<a href="https://colab.research.google.com/github/dorinhazan/FinalProject-DataScience/blob/main/classification_grouping_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openai
import zipfile
import json
import os

In [ ]:
!pip install openai

In [ ]:
!pip install --upgrade openai

In [ ]:
openai.api_key = ""

In [ ]:
# 1. Update these paths
zip_path        = '/content/json_results.zip'
extract_dir     = 'extracted_jsons'
merged_json_path = 'merged_results.json'

# 2. Extract JSON files
with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall(extract_dir)

# 3. Merge into one dictionary, extending duplicates’ observables
merged = {}
for filename in os.listdir(extract_dir):
    # only process .json files
    if not filename.lower().endswith('.json'):
        continue

    full_path = os.path.join(extract_dir, filename)
    with open(full_path, 'r') as f:
        data = json.load(f)

    for md_name, doc in data.items():
        if md_name not in merged:
            merged[md_name] = doc
        else:
            # extend observables list from both
            merged[md_name]["observables"].extend(doc.get("observables", []))
            # you can merge other fields here if needed

# 4. Save the merged JSON
with open(merged_json_path, 'w') as f:
    json.dump(merged, f, indent=2)


In [ ]:

# 5. Extract, lowercase, and dedupe classification values
unique_classifications = {
    obs['classification'].lower()
    for doc in merged.values()
    for obs in doc.get('observables', [])
    if 'classification' in obs
}

In [ ]:
gross_classifications = set(["Reconnaissance",
  "Resource Development",
  "Initial Access",
  "Execution",
  "Persistence",
  "Privilege Escalation",
  "Defense Evasion",
  "Credential Access",
  "Discovery",
  "Lateral Movement",
  "Collection",
  "Command and Control",
  "Exfiltration",
  "Impact"])

In [ ]:
# 3. Build the prompt
prompt = f"""
I am a cybersecurity analyst. I have the following set of unique classification values of observables:

{sorted(unique_classifications)}

Please assign each of these classification values to exactly one of the MITRE ATT&CK® Enterprise tactic categories (the “gross classifications”):

{sorted(gross_classifications)}

If a value does not belong to any of these tactics, assign it under a new category called "Other – new".

Respond with a single JSON object where each key is a tactic name (or "Other – new") and the value is a list of the classification strings assigned to it.
"""


In [ ]:
!pip install openai==0.28

In [ ]:
# 4. Call ChatGPT with model o3-mini-high
response = openai.ChatCompletion.create(
    model="o3-mini-2025-01-31",
    messages=[
        {"role": "system", "content": "You are a helpful cybersecurity expert."},
        {"role": "user", "content": prompt}
    ],
)

# 5. Print the JSON response
print(response.choices[0].message.content)